In [1]:
import numpy as np
import pandas as pd
import joblib
from pandas import DataFrame
import xlsxwriter

from sklearn.impute import KNNImputer
#Get base from IBIS Data to fit
data = pd.ExcelFile("KNN Imputation.xlsx") 

ibis_sa_sheet = data.sheet_names[0]


sa_data = data.parse(ibis_sa_sheet)

data_features_sa = sa_data.loc[:, sa_data.columns] 
data_features_sa = data_features_sa.drop(['ROI'], axis=1)
#Already dropped '1SA-11142','1SA-12142','2SA-11142','2SA-12142'

print(data_features_sa.shape)

(290, 296)


In [2]:
from sklearn.model_selection import train_test_split
#Create code to erase half of the set for training data. Keep the erased half for MAE
#Split for a validation set
#Validation set created from IBIS data
X_train, X_validation_truth = train_test_split(data_features_sa, test_size=0.10, random_state=42)
print('X_train shape {}'.format(X_train.shape))
print('X_validation_truth shape {}'.format(X_validation_truth.shape))
X_validation_1y = X_validation_truth.loc[:, X_validation_truth.columns]
X_validation_1y.drop(X_validation_1y.iloc[:, 0:148], axis=1, inplace=True) #Missing 1 yr
X_validation_2y = X_validation_truth.loc[:, X_validation_truth.columns]
X_validation_2y.drop(X_validation_2y.iloc[:, 148:296], axis=1, inplace=True) #Missing 2 yr

print(X_validation_1y.shape)
print(X_validation_2y.shape)
#Add back NaNs
for i in range(0,148):
    X_validation_2y.insert(i+148,i+148, np.nan) #Add nans for missing 2yr
    X_validation_1y.insert(i,i, np.nan) #Add nans for missing 1yr

print('X_validation_1y shape {}'.format(X_validation_1y.shape))
print('X_validation_2y shape {}'.format(X_validation_2y.shape))

sa_imputer = KNNImputer(n_neighbors=50)
#Fit to IBIS Data
sa_imputer.fit(X_train)

imputed_validation_2y = sa_imputer.transform(X_validation_2y) #Predict 2yr
imputed_validation_1y = sa_imputer.transform(X_validation_1y) #Predict 1yr

#MAE Calculation
from sklearn.metrics import mean_absolute_error
print('MAE 1y Validation KNN {} '.format(mean_absolute_error(imputed_validation_1y, X_validation_truth.to_numpy())))
print('MAE 2y Validation KNN {} '.format(mean_absolute_error(imputed_validation_2y, X_validation_truth.to_numpy())))

X_train shape (261, 296)
X_validation_truth shape (29, 296)
(29, 148)
(29, 148)
X_validation_1y shape (29, 296)
X_validation_2y shape (29, 296)
MAE 1y Validation KNN 0.05068873354307801 
MAE 2y Validation KNN 0.05273130375450548 


In [3]:
#Get the Gilmore data we want imputed to calculate MAE
#'1SA-11142','1SA-12142','2SA-11142','2SA-12142' Already dropped
gilmore_sa_sheet_1yToImpute2y = data.sheet_names[1]
gilmore_sa_sheet_2yToImpute1y = data.sheet_names[2]

gilmore_sa1y_data = data.parse(gilmore_sa_sheet_1yToImpute2y)
gilmore_data_features_sa1y = gilmore_sa1y_data.loc[:, gilmore_sa1y_data.columns]
gilmore_data_features_sa1y = gilmore_data_features_sa1y.drop(['ROI'], axis=1)

gilmore_sa2y_data = data.parse(gilmore_sa_sheet_2yToImpute1y)
gilmore_data_features_sa2y = gilmore_sa2y_data.loc[:, gilmore_sa2y_data.columns]
gilmore_data_features_sa2y = gilmore_data_features_sa2y.drop(['ROI'], axis=1)

print("Individual data feature shapes")
print(gilmore_data_features_sa1y.shape)
print(gilmore_data_features_sa2y.shape)

Individual data feature shapes
(129, 296)
(129, 296)


In [4]:
#Predict Gilmore Data for MAE
imputed_sa2y = sa_imputer.transform(gilmore_data_features_sa1y)
imputed_sa1y = sa_imputer.transform(gilmore_data_features_sa2y)
print("Imputed Individual Shapes")
print(imputed_sa2y.shape)
print(imputed_sa1y.shape)

#DFs for Individuals
df_sa_2y = pd.DataFrame(imputed_sa2y)
df_sa_1y = pd.DataFrame(imputed_sa1y)

df_sa_2y.to_excel("Imputed SA 2y KNN.xlsx", index=False)
df_sa_1y.to_excel("Imputed SA 1y KNN.xlsx", index=False)

#From here calculate MAE to see which imputation method within KNN works better.
#Ground Truth
gilmore_sa_sheet = data.sheet_names[3]

gilmore_sa1y_data = data.parse(gilmore_sa_sheet)
gilmore_data_features_sa1y = gilmore_sa1y_data.loc[:, gilmore_sa1y_data.columns]
gilmore_data_features_sa1y = gilmore_data_features_sa1y.drop(['ROI'], axis=1)
gilmore_data_features_sa1y.drop(gilmore_data_features_sa1y.iloc[:, 148:296], axis=1, inplace=True)
print(gilmore_data_features_sa1y)

gilmore_sa2y_data = data.parse(gilmore_sa_sheet)
gilmore_data_features_sa2y = gilmore_sa2y_data.loc[:, gilmore_sa2y_data.columns]
gilmore_data_features_sa2y = gilmore_data_features_sa2y.drop(['ROI'], axis=1)
gilmore_data_features_sa2y.drop(gilmore_data_features_sa2y.iloc[:, 0:147], axis=1, inplace=True)
print(gilmore_data_features_sa2y)

#Imputed
df_imputed_sa_1y_xlsx = pd.ExcelFile("Imputed SA 1y KNN.xlsx")
imputed_sa_sheet = df_imputed_sa_1y_xlsx.sheet_names[0]
imputed_sa_data = df_imputed_sa_1y_xlsx.parse(imputed_sa_sheet)
imputed_data_features_sa1y = imputed_sa_data.loc[:, imputed_sa_data.columns]
imputed_data_features_sa1y.drop(imputed_data_features_sa1y.iloc[:, 148:296], axis=1, inplace=True)
print(imputed_data_features_sa1y)

df_imputed_sa_2y_xlsx = pd.ExcelFile("Imputed SA 2y KNN.xlsx")
imputed_sa_sheet = df_imputed_sa_2y_xlsx.sheet_names[0]
imputed_sa_data = df_imputed_sa_2y_xlsx.parse(imputed_sa_sheet)
imputed_data_features_sa2y = imputed_sa_data.loc[:, imputed_sa_data.columns]
imputed_data_features_sa2y.drop(imputed_data_features_sa2y.iloc[:, 0:147], axis=1, inplace=True)
print(imputed_data_features_sa2y)

from sklearn.metrics import mean_absolute_error
print('MAE 1y KNN {} '.format(mean_absolute_error(gilmore_data_features_sa1y.to_numpy(), imputed_data_features_sa1y.to_numpy())))
print('MAE 2y KNN {} '.format(mean_absolute_error(gilmore_data_features_sa2y.to_numpy(), imputed_data_features_sa2y.to_numpy())))

#Remove extra features for MAE calculation - Done
#Find Optimal number of neighbors - todo
#Create a validation set for KNN to see if the trend for AC having high MAE continues - todo
#N = 2
#MAE 1y KNN 0.16280582283255568 
#MAE 2y KNN 0.16321320699559033 
#N = 3
#MAE 1y KNN 0.1577243075443826 
#MAE 2y KNN 0.15720122874457262 
#N = 4
#MAE 1y KNN 0.15519481307686972 
#MAE 2y KNN 0.15487959605664114 
#N = 5
#MAE 1y KNN 0.15356291288903134 
#MAE 2y KNN 0.15294622209838285 
#N = 10
#MAE 1y KNN 0.15003403232716447 
#MAE 2y KNN 0.14948200320277555 
#N = 25
#MAE 1y KNN 0.14792755622564427 
#MAE 2y KNN 0.14750929111677524 
#N = 50
#MAE 1y KNN 0.14757113789488976 
#MAE 2y KNN 0.1475000914221722 
#N = 100
#MAE 1y KNN 0.14826055109796843 
#MAE 2y KNN 0.1483661596808016 

Imputed Individual Shapes
(129, 296)
(129, 296)
     1SA-11101  1SA-11102  1SA-11103  1SA-11104  1SA-11105  1SA-11106  \
0     0.389491   0.552493   0.340734   0.540838   0.319828   0.522882   
1     0.221516   0.578360   0.348207   0.379920   0.628914   0.438039   
2     0.239695   0.373045   0.393017   0.357912   0.393315   0.255545   
3     0.349985   0.465013   0.231390   0.489339   0.460911   0.316039   
4     0.576608   0.358640   0.344260   0.634978   0.430610   0.738359   
..         ...        ...        ...        ...        ...        ...   
124   0.191075   0.698679   0.064813   0.297003   0.513894   0.328628   
125   0.131991   0.444319   0.111981   0.230278   0.229674   0.196791   
126   0.273170   0.476426   0.125102   0.361693   0.229266   0.264249   
127   0.226456   0.517992   0.538361   0.189651   0.586598   0.214520   
128   0.314758   0.583183   0.393826   0.508951   0.772201   0.859330   

     1SA-11107  1SA-11108  1SA-11109  1SA-11110  ...  1SA-12166  1SA-12167 

          147       148       149       150       151       152       153  \
0    0.156133  0.585257  0.534765  0.291406  0.515869  0.447556  0.534687   
1    0.000000  0.557433  0.532716  0.271292  0.466256  0.434887  0.532195   
2    0.468932  0.532340  0.524536  0.251892  0.446553  0.415206  0.511827   
3    0.759886  0.539936  0.518101  0.288750  0.531247  0.481673  0.549832   
4    0.840500  0.570267  0.560859  0.307970  0.541521  0.499067  0.608855   
..        ...       ...       ...       ...       ...       ...       ...   
124  0.507116  0.516628  0.540905  0.233044  0.409749  0.441122  0.473190   
125  0.322406  0.534288  0.534904  0.251256  0.420248  0.409759  0.452916   
126  0.386537  0.529407  0.528939  0.254852  0.426196  0.427644  0.456303   
127  0.458770  0.527868  0.486767  0.241499  0.396363  0.420536  0.444252   
128  0.672940  0.545440  0.539376  0.300223  0.515782  0.468299  0.606979   

          154       155       156  ...       286       287       288  \
0  

In [5]:
#Get the data we want imputed
interpolate_data = pd.ExcelFile("Data to be Interpolated.xlsx") 
sa1y = interpolate_data.sheet_names[4]
sa2y = interpolate_data.sheet_names[9]

sa1y_data = interpolate_data.parse(sa1y)
data_features_sa1y = sa1y_data.loc[:, sa1y_data.columns] 
data_features_sa1y = data_features_sa1y.drop(['ROI'], axis=1)

sa2y_data = interpolate_data.parse(sa2y)
data_features_sa2y = sa2y_data.loc[:, sa2y_data.columns] 
data_features_sa2y = data_features_sa2y.drop(['ROI'], axis=1)

print(data_features_sa1y.shape)
print(data_features_sa2y.shape)

(36, 296)
(20, 296)


In [11]:
#Predict the data we want to impute here for the downstream task
predicted_sa2y = sa_imputer.transform(data_features_sa1y)
df_predicted_sa2y = pd.DataFrame(predicted_sa2y)
df_predicted_sa2y.to_excel("Interpolated SA2y Downstream.xlsx", index=False)

predicted_sa1y = sa_imputer.transform(data_features_sa2y)
df_predicted_sa1y = pd.DataFrame(predicted_sa1y)
df_predicted_sa1y.to_excel("Interpolated SA1y Downstream.xlsx", index=False)